In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['PKL']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [8]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 0]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [9]:
list(zip(longforms, counts))

[('pickle', 30),
 ('paxillin kinase linker', 11),
 ('pyruvate kinase liver isoform', 2),
 ('pine kraft lignin', 1),
 ('pinus koraiensis leaf', 1),
 ('pkl', 1),
 ('protein kinase like', 1),
 ('pyruvate kinase l', 1),
 ('pyruvate kinase liver', 1),
 ('primary head kidney leucocytes', 1),
 ('liver type pk', 1),
 ('pkl mutant', 1),
 ('pickle gymnos', 1),
 ('pkl wild type', 1),
 ('trp53floxflox krasg12d + rosa26lsl luciferase lsl luciferase', 1)]

In [12]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

ERROR: [2020-10-28 20:13:11] adeft.gui.ground - Exception on /ground_add [POST]
Traceback (most recent call last):
  File "/adeft/.virtualenvs/adeft_env/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/adeft/.virtualenvs/adeft_env/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/adeft/.virtualenvs/adeft_env/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/adeft/.virtualenvs/adeft_env/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/adeft/.virtualenvs/adeft_env/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/adeft/.virtualenvs/adeft_env/lib/python3.8/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint

In [13]:
result = [grounding_map, names, pos_labels]

In [14]:
result

[{'liver type pk': 'HGNC:9020',
  'paxillin kinase linker': 'HGNC:4273',
  'pickle': 'UP:Q9S775',
  'pickle gymnos': 'ungrounded',
  'pine kraft lignin': 'ungrounded',
  'pinus koraiensis leaf': 'ungrounded',
  'pkl': 'ungrounded',
  'pkl mutant': 'ungrounded',
  'pkl wild type': 'ungrounded',
  'primary head kidney leucocytes': 'ungrounded',
  'protein kinase like': 'ungrounded',
  'pyruvate kinase l': 'HGNC:9020',
  'pyruvate kinase liver': 'HGNC:9020',
  'pyruvate kinase liver isoform': 'HGNC:9020',
  'trp53floxflox krasg12d + rosa26lsl luciferase lsl luciferase': 'ungrounded'},
 {'HGNC:9020': 'PKLR',
  'HGNC:4273': 'GIT2',
  'UP:Q9S775': 'CHD3-type chromatin-remodeling factor PICKLE'},
 ['HGNC:4273', 'HGNC:9020', 'UP:Q9S775']]

In [10]:
grounding_map, names, pos_labels = [{'liver type pk': 'HGNC:9020',
  'paxillin kinase linker': 'HGNC:4273',
  'pickle': 'UP:Q9S775',
  'pickle gymnos': 'UP:Q9S775',
  'pine kraft lignin': 'ungrounded',
  'pinus koraiensis leaf': 'ungrounded',
  'pkl': 'ungrounded',
  'pkl mutant': 'ungrounded',
  'pkl wild type': 'ungrounded',
  'primary head kidney leucocytes': 'ungrounded',
  'protein kinase like': 'ungrounded',
  'pyruvate kinase l': 'HGNC:9020',
  'pyruvate kinase liver': 'HGNC:9020',
  'pyruvate kinase liver isoform': 'HGNC:9020',
  'trp53floxflox krasg12d + rosa26lsl luciferase lsl luciferase': 'ungrounded'},
 {'HGNC:9020': 'PKLR',
  'HGNC:4273': 'GIT2',
  'UP:Q9S775': 'CHD3-type chromatin-remodeling factor PICKLE'},
 ['HGNC:4273', 'HGNC:9020', 'UP:Q9S775']]

In [15]:
excluded_longforms = ['pkl', 'pkl mutant', 'pkl wild type']

In [16]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [17]:
additional_entities = {}

In [18]:
unambiguous_agent_texts = {}

In [19]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [12]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [13]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [14]:
intersection1

[('HGNC:1848', 'HGNC:1848', 86)]

In [15]:
intersection2

[('MESH:D018893', 'HGNC:1848', 0),
 ('HGNC:1848', 'HGNC:1848', 1),
 ('ungrounded', 'HGNC:1848', 0),
 ('CHEBI:CHEBI:64198', 'HGNC:1848', 0),
 ('MESH:D019164', 'HGNC:1848', 0),
 ('MESH:C059416', 'HGNC:1848', 0),
 ('CHEBI:CHEBI:15710', 'HGNC:1848', 0)]

In [17]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [18]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [20]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-28 20:15:01] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-28 20:15:03] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.827418414918415 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [21]:
classifier.stats

{'label_distribution': {'HGNC:9020': 4,
  'HGNC:4273': 9,
  'UP:Q9S775': 27,
  'ungrounded': 6},
 'f1': {'mean': 0.827418, 'std': 0.092212},
 'precision': {'mean': 0.775149, 'std': 0.122412},
 'recall': {'mean': 0.9, 'std': 0.05},
 'HGNC:9020': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'HGNC:4273': {'f1': {'mean': 1.0, 'std': 0.0},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'ungrounded': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'UP:Q9S775': {'f1': {'mean': 0.886946, 'std': 0.079131},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.805952, 'std': 0.128196}}}

In [22]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [23]:
disamb.dump(model_name, results_path)

In [24]:
print(disamb.info())

Disambiguation model for PKL

Produces the disambiguations:
	CHD3-type chromatin-remodeling factor PICKLE*	UP:Q9S775
	GIT2*	HGNC:4273
	PKLR*	HGNC:9020

Class level metrics:
--------------------
Grounding                                   	Count	F1     
CHD3-type chromatin-remodeling factor PICKLE*	27	0.88695
                                        GIT2*	 9	    1.0
                                  Ungrounded	 6	    0.0
                                        PKLR*	 4	    0.0

Weighted Metrics:
-----------------
	F1 score:	0.82742
	Precision:	0.77515
	Recall:		0.9

* Positive labels
See Docstring for explanation



In [25]:
model_to_s3(disamb)